In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
new_df = pd.read_excel("/Users/gayanmeerigama/Creative Cloud Files/Data Analytics Boot Camp/Project_3/AiVsHumanNew.xlsx")

In [3]:
new_df.groupby('ai_generated').count()

,Unnamed: 0.1,Unnamed: 0,essay_id,essay_set,essay,HumanVsAi
ai_generated,,,,,,
Ai-generated,2967,2967,2967,2967,2967,2967
Human-generated,2967,2967,2967,2967,2967,2967


In [4]:
new_df

,Unnamed: 0.1,Unnamed: 0,essay_id,essay_set,essay,HumanVsAi,ai_generated
0,18483,1287,8381,3,The features of the setting affected the cycle...,human-generated,Human-generated
1,18484,1288,8382,3,Many things affected the cyclist in this story...,human-generated,Human-generated
2,18485,1289,8383,3,There are three very big features of the setti...,human-generated,Human-generated
3,18486,1290,8384,3,The setting affected the cyclist beause if he ...,human-generated,Human-generated
4,18487,1291,8385,3,In the story Do Not Exceed posted Speed Limit...,human-generated,Human-generated
...,...,...,...,...,...,...,...
5929,24412,213,213,1,"\n\nDear Editor, \n\nAs a citizen of this comm...",Ai-generated,Ai-generated
5930,24413,214,214,1,"\n\n\nDear Editor,\n\nAs a concerned citizen a...",Ai-generated,Ai-generated
5931,24414,215,215,1,"\n\nDear Editor,\n\nI am writing to share my o...",Ai-generated,Ai-generated
5932,24415,216,216,1,\n\nTo the Editor: \n\nAs our world has become...,Ai-generated,Ai-generated


In [5]:
#Removing Special Charactes 
new_df["Remove_Special_Characters"]=new_df["essay"].str.replace('\W', ' ', regex=True)
new_df_R=new_df["Remove_Special_Characters"]
new_df_R

0       The features of the setting affected the cycle...
1       Many things affected the cyclist in this story...
2       There are three very big features of the setti...
3       The setting affected the cyclist beause if he ...
4       In the story  Do Not Exceed posted Speed Limit...
                              ...                        
5929      Dear Editor    As a citizen of this communit...
5930       Dear Editor   As a concerned citizen and lo...
5931      Dear Editor   I am writing to share my opini...
5932      To the Editor    As our world has become inc...
5933      Dear Editor    I am writing to present my th...
Name: Remove_Special_Characters, Length: 5934, dtype: object

In [6]:
# lemmatization is a lot more powerful. It looks beyond word reduction and considers a language’s full vocabulary to
# apply a morphological analysis to words, aiming to remove inflectional endings only and to return the base or 
# dictionary form of a word
# Wordnet is a publicly available lexical database of over 200 languages that provides semantic relationships betweenits words

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')


def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(new_df_R):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(new_df_R))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

# Lemmatizing
new_df["Lemmatizing"] = new_df_R.apply(lambda x: lemmatize_sentence(x))

In [7]:
#The words which are generally filtered out before processing a natural language are called stop words
#Examples of a few stop words in English are “the”, “a”, “an”, “so”, “what”.
#NLTK is a library to play with natural language.The steps to import the library and the English stop words list
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
new_df['without_stopwords'] = new_df["Lemmatizing"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(new_df['without_stopwords'][1])
print("Old length: ", len(new_df["Lemmatizing"][1]))
print("New length: ", len(new_df['without_stopwords'][1]))

Many thing affect cyclist story The heat rlack water hill wild life affect The heat cause cyclist become right head grow realization I could drop heatstroke He hot cyclist afraid heatstroke make sure mirage The heat also affect visibility thinking Whe first saw factory worry mirage see thing Also lack water make dehydrate It make body hard function I ease past try keep balance dehydrated state The lack water make hard ride bike That smoky blue tune rattle around dry honey comb deteriorate brain The heat lack water make hard think straight The deserted desert impare situation He lose one help barren wasteland If people around could ask help In end setting break find filth bait shop The featured setting help hinder cyclist journey
Old length:  1069
New length:  738


In [8]:
new_df

,Unnamed: 0.1,Unnamed: 0,essay_id,essay_set,essay,HumanVsAi,ai_generated,Remove_Special_Characters,Lemmatizing,without_stopwords
0,18483,1287,8381,3,The features of the setting affected the cycle...,human-generated,Human-generated,The features of the setting affected the cycle...,The feature of the setting affect the cycleist...,The feature setting affect cycleist many way F...
1,18484,1288,8382,3,Many things affected the cyclist in this story...,human-generated,Human-generated,Many things affected the cyclist in this story...,Many thing affect the cyclist in this story Th...,Many thing affect cyclist story The heat rlack...
2,18485,1289,8383,3,There are three very big features of the setti...,human-generated,Human-generated,There are three very big features of the setti...,There be three very big feature of the setting...,There three big feature setting affect cyclist...
3,18486,1290,8384,3,The setting affected the cyclist beause if he ...,human-generated,Human-generated,The setting affected the cyclist beause if he ...,The setting affect the cyclist beause if he be...,The setting affect cyclist beause friendly coo...
4,18487,1291,8385,3,In the story Do Not Exceed posted Speed Limit...,human-generated,Human-generated,In the story Do Not Exceed posted Speed Limit...,In the story Do Not Exceed post Speed Limit th...,In story Do Not Exceed post Speed Limit cyclis...
...,...,...,...,...,...,...,...,...,...,...
5929,24412,213,213,1,"\n\nDear Editor, \n\nAs a citizen of this comm...",Ai-generated,Ai-generated,Dear Editor As a citizen of this communit...,Dear Editor As a citizen of this community I f...,Dear Editor As citizen community I feel import...
5930,24413,214,214,1,"\n\n\nDear Editor,\n\nAs a concerned citizen a...",Ai-generated,Ai-generated,Dear Editor As a concerned citizen and lo...,Dear Editor As a concerned citizen and long ti...,Dear Editor As concerned citizen long time res...
5931,24414,215,215,1,"\n\nDear Editor,\n\nI am writing to share my o...",Ai-generated,Ai-generated,Dear Editor I am writing to share my opini...,Dear Editor I be write to share my opinion on ...,Dear Editor I write share opinion effect compu...
5932,24415,216,216,1,\n\nTo the Editor: \n\nAs our world has become...,Ai-generated,Ai-generated,To the Editor As our world has become inc...,To the Editor As our world have become increas...,To Editor As world become increasingly reliant...


In [9]:
# Seperate the features, X,  from the target variable, y
y = new_df['HumanVsAi']
X = new_df['without_stopwords']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Builds a dictionary of features and transforms documents to feature vectors and convert our text documents to a
# matrix of token counts (CountVectorizer)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# When use tf.fit_transform() on your test / new data, 
# it extracts a new vocabulary based on the words in your test data which are likely different
# than your training data.

# transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [12]:
# creating dataframe from bag of words matrix representation
words_matrix = pd.DataFrame(X_train_counts.toarray(), columns = count_vect.get_feature_names_out())
words_matrix

,00,000,046,10,100,1000,101st,102,102nd,103,...,zing,zip,ziplining,zipper,zone,zoo,zoom,zooooooooom,zu,zuruck
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#To reduce document length bias, you use normalization as you proportionally scale each term's Tfidf score based on 
# total score of that document (simple average for norm=l1, squared average for norm=l2)
normalized_tf_idf = pd.DataFrame(X_train_tfidf.toarray(), columns = count_vect.get_feature_names_out())
normalized_tf_idf

,00,000,046,10,100,1000,101st,102,102nd,103,...,zing,zip,ziplining,zipper,zone,zoo,zoom,zooooooooom,zu,zuruck
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
knn = KNeighborsClassifier(n_neighbors=2)

# training our classifier ; new_df['essay'] will be having numbers assigned for each category in train data
clf = knn.fit(X_train_tfidf,y_train)

# Input Data to predict their classes of the given categories

#Ai-generated
docs_new = ['Technology has revolutionized various aspects of our lives, and one domain that has witnessed significant transformation is education. The integration of technology in educational settings has redefined the learning experience, enhancing access to information, fostering interactive engagement, and promoting personalized instruction. In this essay, we will explore the profound impact of technology on education, including its role in expanding educational opportunities, improving teaching methods, and preparing students for the demands of the 21st century']
#human-generated
#docs_new = ['Many thing affect cyclist story The heat rlack water hill wild life affect The heat cause cyclist become right head grow realization I could drop heatstroke He hot cyclist afraid heatstroke make sure mirage The heat also affect visibility thinking Whe first saw factory worry mirage see thing Also lack water make dehydrate It make body hard function I ease past try keep balance dehydrated state The lack water make hard ride bike That smoky blue tune rattle around dry honey comb deteriorate brain The heat lack water make hard think straight The deserted desert impare situation He lose one help barren wasteland If people around could ask help In end setting break find filth bait shop The featured setting help hinder cyclist journey']
# building up feature vector of our input to chat with friends/new people
X_new_counts = count_vect.transform(docs_new)
# We call transform instead of fit_transform because it's already been fit
X_new_tfidf = tfidf_transformer.transform(X_new_counts)


In [16]:
import joblib
# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(clf, filename)

['finalized_model.sav']

In [20]:
# load the model from disk
loaded_model = joblib.load(filename)


In [57]:
 pd.DataFrame(X_new_counts.toarray(), columns = count_vect.get_feature_names_out())

,00,000,046,10,100,1000,101st,102,102nd,103,...,zing,zip,ziplining,zipper,zone,zoo,zoom,zooooooooom,zu,zuruck
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
pd.DataFrame(X_new_tfidf.toarray(), columns = count_vect.get_feature_names_out())

,00,000,046,10,100,1000,101st,102,102nd,103,...,zing,zip,ziplining,zipper,zone,zoo,zoom,zooooooooom,zu,zuruck
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
predicted=clf.predict(X_new_tfidf)
print(predicted)

['Ai-generated']


In [60]:
# predicting the category of our input text: Will give out number for category
v = dict(zip(list(y),y.to_list()))
for doc, category in zip(docs_new, predicted):
    print('%r ============== Essay is belongs to===> %s' % (doc,v[category]))
   

'Technology has revolutionized various aspects of our lives, and one domain that has witnessed significant transformation is education. The integration of technology in educational settings has redefined the learning experience, enhancing access to information, fostering interactive engagement, and promoting personalized instruction. In this essay, we will explore the profound impact of technology on education, including its role in expanding educational opportunities, improving teaching methods, and preparing students for the demands of the 21st century' ============== Essay is belongs to===> Ai-generated


In [38]:
# We can use Pipeline to add vectorizer -> transformer -> classifier all in a one compound classifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', knn),
])
# Fitting our train data to the pipeline
text_clf.fit(X_train, y_train)
                              
docs_test = X_test
# Predicting our test data
predicted = text_clf.predict(docs_test)
print('We got an accuracy of',np.mean(predicted ==  y_test)*100, '% over the test data.')

We got an accuracy of 97.911051212938 % over the test data.
